In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


LOG.propagate = False

Lab7 - Kalman Filter

In [1]:
# Get ArtemisBLEController object
ble = get_ble_controller()
# Connect to the Artemis Device
ble.connect()

NameError: name 'get_ble_controller' is not defined

In [ ]:
timeData = []
tof1Data = []
tof2Data = []
leftPWM = []
rightPWM = []
errorData = []
p_data = []
i_data = []
d_data = []

In [ ]:
def pid_fd_notification_handler(uuid, char_bytearr):
    global timeData, tof1Data, tof2Data, leftPWM, rightPWM, errorData, p_data, i_data, d_data
    string = ble.bytearray_to_string(char_bytearr)
    parts = string.split("|")  
    print(parts)
    timeData.append(int(parts[0][2:]))
    tof1Data.append(float(parts[1][6:]))
    tof2Data.append(float(parts[2][6:]))
    leftPWM.append(float(parts[3][8:]))
    rightPWM.append(float(parts[4][9:]))
    errorData.append(float(parts[5][10:]))
    p_data.append(float(parts[6][7:]))
    i_data.append(float(parts[7][7:]))
    d_data.append(float(parts[8][7:]))
    

In [ ]:
ble.start_notify(ble.uuid["RX_STRING"], pid_fd_notification_handler)

In [ ]:
ble.send_command(CMD.PID_STATUS_INI, "1")

In [ ]:
ble.send_command(CMD.KALMAN_FILTER_TEST, "")

In [ ]:
ble.send_command(CMD.SEND_PID_FD_DEBUG_READINGS, "")

In [ ]:
# # Define the filename 
# filename = "/Users/harrypeng/Desktop/ECE4160/lab1/ble_robot-1.1/ble_python/kalman_readings_csv/time_dist_2024-03-20_12-16-20.csv"

# # Load the data from the CSV file
# loaded_data = np.loadtxt(filename, delimiter=',', skiprows=1)  # skip the header row

# # Split the loaded data into two arrays
# timeData = np.array(loaded_data[:, 0])  # unit: s
# tof1Data = np.array(loaded_data[:, 1])/1000  # unit: m


In [ ]:
# # change unit
timeData = np.array(timeData) / 1000    # unit: s
tof1Data = np.array(tof1Data) / 1000    # unit: m

# timeData = np.array(timeData)   
# tof1Data = np.array(tof1Data)   

In [ ]:
print(len(timeData))
print(len(tof1Data))
print(len(leftPWM))
print(len(p_data))
print(len(i_data))

In [ ]:
unused_index_arr = np.argwhere(np.array(timeData)==0)
if (len(unused_index_arr) != 0):
    unused_index = unused_index_arr[0][0]
else:
    unused_index = len(timeData)
print(unused_index)

In [ ]:
# plot distance vs time
plt.figure(figsize=(15, 10))
plt.plot(timeData[:unused_index], tof1Data[:unused_index], linestyle='-', marker='o', color='blue', label='distance')
plt.title("distance vs time")
plt.xlabel("time(s)")
plt.ylabel("distance(m)")
plt.show()


In [ ]:
# plot velocity vs time
##### use three points to calculate the middle point's velocity
start_index = np.argwhere(np.array(timeData)>2)[0][0]
velocityData = []
for i in range(1, (unused_index-1)):
    if (i < start_index):
        velocityData.append(0)
    else:
        velocityData.append(np.abs((tof1Data[i-1]-tof1Data[i+1])/(timeData[i+1] - timeData[i-1])))

velocityData = np.array(velocityData)
final_velocity = np.max(velocityData)
steady_state_velocity = round(0.9*final_velocity, 3)

rise_time_with_delay = timeData[np.argwhere(velocityData>=0.9*final_velocity)[0][0]]
rise_time = rise_time_with_delay - 2 # deduct the first 2 sec delay

plt.axvline(x=rise_time_with_delay, color='red', linestyle='--', label=f"rise time with delay:{rise_time_with_delay}s")
plt.plot(timeData[1:unused_index-1], velocityData, linestyle='-', marker='o', color='green', label='velocity')
plt.plot(timeData[:unused_index-1], np.ones(np.shape(timeData[:unused_index-1]))*steady_state_velocity, linestyle='--', color='black', label=f'velocity at 90%:{steady_state_velocity}m/s')
plt.title("velocity vs time")
plt.xlabel("time(s)")
plt.ylabel("velocity(mm/s)")
plt.legend()
plt.show()

In [ ]:
# plot pwm vs time
plt.plot(timeData[:unused_index], leftPWM[:unused_index], linestyle='-', marker='o', color='red', label='pwm')
plt.title("pwm vs time")
plt.xlabel("time(s)")
plt.ylabel("pwm")
plt.show()

In [ ]:
# save the readings data
# Stack the arrays vertically
data = np.vstack((timeData, tof1Data))

# Get the current date and time
now = datetime.now()

# Format the date and time as a string
date_time_str = now.strftime("%Y-%m-%d_%H-%M-%S")

# Set float format to avoid scientific notation
float_format = "%.8f"

# Create the filename with the date and time
filename = f"/Users/harrypeng/Desktop/ECE4160/lab1/ble_robot-1.1/ble_python/kalman_readings_csv/time_dist_{date_time_str}.csv"

# Save the data to a CSV file
np.savetxt(filename, data.T, delimiter=",", header="timeData,tof1Data", comments="", fmt=float_format)


In [ ]:
print(steady_state_velocity)
print(rise_time)

In [ ]:
# calculate d and m

drag = 1 / steady_state_velocity

momentum = -drag * rise_time / np.log(1-0.9)

print("drag = ", drag, "\tmomentum = ", momentum)

In [ ]:
### average the d and m
drag1 = 0.8375209380234506
momentum1 = 1.4287342754740142

drag2 = 0.7739938080495355
momentum2 = 1.2874209486760482

avg_drag = (drag1 + drag2) / 2
avg_momentum = (momentum1 + momentum2) / 2

print(f"average drag = {avg_drag} \taverage momentum = {avg_momentum}")

**KF Initialization**

In [ ]:
n = 2
dt = timeData[1] - timeData[0]
A = np.array([[0, 1], [0, -avg_drag/avg_momentum]])
B = np.array([[0], [1/avg_momentum]])
C = np.array([[-1, 0]])
Ad = np.eye(n) + dt * A
Bd = dt * B
print(dt)

Noise Initializaition

In [ ]:
sigma_1 = 0.02  # trust in modeled position
sigma_2 = 0.1   # trust in modeled velocity
sigma_3 = 0.02  # trust in measurement 
Sigma_z = np.array([[sigma_3**2]])
Sigma_u = np.array([[sigma_1**2, 0], [0, sigma_2**2]])

In [ ]:
# mu -> state estimate
# sigma -> state uncertainty
# u -> input
# y -> state from measurement
def kf (mu, sigma, u, y, Sigma_u, Sigma_z) :
    mu_p = np.dot(Ad, mu) + np.dot(Bd, u)
    sigma_p = np.dot(Ad, np.dot(sigma, np.transpose(Ad))) + Sigma_u
    sigma_m = np.dot(C, np.dot(sigma_p, np.transpose(C))) + Sigma_z
    kkf_gain = np.dot(sigma_p, np.dot(np.transpose(C), np.linalg.inv(sigma_m)))
    y_m = y - np.dot(C, mu_p)
    mu = mu_p + np.dot(kkf_gain, y_m)
    sigma = np.dot((np.eye(2) - np.dot(kkf_gain, C)), sigma_p)
    return mu, sigma

In [ ]:
# Define the filename 
filename = "/Users/harrypeng/Desktop/ECE4160/lab1/ble_robot-1.1/ble_python/kalman_readings_csv/time_pwm_distance_2024-03-20_19-22-01.csv"

# Load the data from the CSV file
loaded_data = np.loadtxt(filename, delimiter=',', skiprows=1)  # skip the header row

# Split the loaded data into two arrays
timeData = np.array(loaded_data[:, 0])/1000  # unit: s
pwm = np.array(loaded_data[:, 1])/120        # scaled PWM values
distanceData = np.array(loaded_data[:, 2]) / 1000   # unit: m


In [ ]:
## initial state
x = np.array([[-distanceData[0]], [0]])
sigma = np.array([[sigma_3, 0], [0, sigma_3]])

kf_prediction = []
for i in range(len(timeData)):
    x, sigma = kf(x, sigma, [[pwm[i]]], [[-distanceData[i]]], Sigma_u, Sigma_z)
    kf_prediction.append(x[0][0])

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(timeData, distanceData, linestyle='-', color='red', label='ToF Dist')
plt.plot(timeData, np.array(kf_prediction), linestyle='-', color='blue', label='KF Pred')
plt.legend()
plt.show()


## Disconnect

In [ ]:
# Disconnect
ble.disconnect()